In [76]:
from copy import deepcopy

file = open("../Data/input_03.txt", "r")
content = file.read().split('\n')[:-1]
file.close()

# symbols = {}

# for el in list(''.join(content)):
#     symbols[el] = 1

# print(symbols.keys())

# relavent symbols
search_symbols = ['*', '=', '/', '%', '#', '&', '$', '-', '@', '+']


# create a mask map   [search symbol] -> 1   [. 0-9] -> 0
symbol_mask = []

for line in content:
    line_mask = []

    for c in list(line):
        if c in search_symbols:
            line_mask.append(1)

        else:
            line_mask.append(0)

    symbol_mask.append(line_mask)
    

def mask_expansion(mask, i, j, last_index):

    mask[i][j] = 1

    # not first element
    if j != 0:
        mask[i][j-1] = 1

    # not last element
    if j != last_index:
        mask[i][j+1] = 1

    return mask


symbol_mask_exp = deepcopy(symbol_mask)

for i, line_mask in enumerate(symbol_mask):
    
    for j, el in enumerate(line_mask):
        
        last_index = len(line_mask)-1

        if el == 1:
            
            symbol_mask_exp = mask_expansion(symbol_mask_exp, i, j, last_index)

            # not first line
            if i != 0:
                symbol_mask_exp = mask_expansion(symbol_mask_exp, i-1, j, last_index)

            # not last line
            if i != 0:
                symbol_mask_exp = mask_expansion(symbol_mask_exp, i+1, j, last_index)




In [78]:
content

['..172..............................454..46.......507..........809......923.778..................793..............137.............238........',
 '............*.........712........=.......*................515.*...........*.......690.........../..........658.........=.........*..........',
 '.........823.835........%.........710.....749........134..%............................#812...&.....925.../..........276.......386..........',
 '519..................13......341.................481....=.....$............-.......211.......92.......*.....................................',
 '............832*105..@........$..................*.........797.....535..932.........*....152...........123.........678.540...........-...6..',
 '....&..948..........................271....-....228..79.26.........................733...=...715............27.586........*.......883...*...',
 '..172.......=..+.............88&....%....340.55.....+.............465..398......=..................585.......*....*812...347....

[[0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0
  0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0
  0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0
  0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0
  0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1
  0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0
  0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0
  0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1
  0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 1 1 1 0

In [74]:
print(symbol_mask_exp[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [68]:
print(symbol_mask[2])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
